In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
import cv2
import numpy as np
import tkinter
from tkinter import messagebox
import smtplib


In [ ]:
#initialize tkinter

root = tkinter.Tk()    #to create root window with basic widgets
root.withdraw()        #to hide this window

''

In [ ]:
#loading the trained DL model

model = load_model('mask_detection.h5')

In [ ]:
#classifier to detect face

face_det_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

In [ ]:
#capture video using webcam
vid_source = cv2.VideoCapture(0)

text_dict = {0 : "MASKED" , 1 : "NO MASK"}
rect_color_dict = {0 : (0,255,0) , 1 : (0,0,255)}

while(vid_source.isOpened()):
    ret , img = vid_source.read()
    grayscale_img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_det_classifier.detectMultiScale(grayscale_img , 1.3 , 5)
    
    for (x,y,w,h) in faces:
        face_img = grayscale_img[y:y+h , x:x+w ]
        resized_img = cv2.resize(face_img ,(112,112))
        normalized_img = resized_img/255.0
        reshaped_img = np.reshape(normalized_img,(1,112,112,1))
        result = model.predict(reshaped_img)
        
        label = np.argmax(result , axis=1)[0]      #argmax method is same as np.predict_classes which will predict the output as class, here our image is predicted by our model
        
        cv2.rectangle(img , (x,y) , (x+w,y+h) , rect_color_dict[label],4)
        #cv2.rectangle(img , (x,y-40) , (x+w,y+h) , rect_color_dict[label],-1)
        cv2.putText(img,text_dict[label] , (x,y-10) , cv2.FONT_HERSHEY_SIMPLEX , 0.8 , (28,252,249) , 2)
        
    cv2.imshow("mask_detection" , img)
    key = cv2.waitKey(1)
    if(key == 27):
        break
cv2.destroyAllWindows()
vid_source.release()
            
        